<a href="https://colab.research.google.com/github/HwangBoSungHun/ESRGAN_video_tf/blob/main/ESRGAN_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import cv2
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

Downloaded https://tfhub.dev/captain-pool/esrgan-tf2/1, Total size: 20.60MB



In [ ]:
video_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/data/a_jewel_in_the_palace.mp4'
lr_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/lr.mp4'
sr1_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/sr1.mp4'
sr2_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/sr2.mp4'
sec = 30 # 저장하고 싶은 영상 길이(30초 분량만 저장)

In [ ]:
vid = cv2.VideoCapture(video_path)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
lr_out = cv2.VideoWriter(, codec, fps, (width//4, height//4)) # lr은 4배로 줄임
sr1_out = cv2.VideoWriter(sr1_path, codec, fps, (width, height)) # lr에 ESRGAN 적용
sr2_out = cv2.VideoWriter(sr2_path, codec, fps, (width*4, height*4)) # 원본 영상에 ESRGAN 적용(사이즈 4배로 커짐)

In [ ]:
def preprocess_image(hr_image):
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4  # height, width를 4의 배수가 되게 만듦
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

In [ ]:
# Defining helper functions
def downscale_image(image):
  """
      Scales down images using bicubic downsampling.
      Args:
          image: 3D or 4D tensor of preprocessed image
  """
  image_size = []
  if len(image.shape) == 3:
    image_size = [image.shape[1], image.shape[0]]
  else:
    raise ValueError("Dimension mismatch. Can work only on single image.")

  image = tf.squeeze(
      tf.cast(
          tf.clip_by_value(image, 0, 255), tf.uint8))

  lr_image = np.asarray(
    Image.fromarray(image.numpy())
    .resize([image_size[0] // 4, image_size[1] // 4],
              Image.BICUBIC))

  lr_image = tf.expand_dims(lr_image, 0)
  lr_image = tf.cast(lr_image, tf.float32)
  return lr_image

In [ ]:
i = 0
while True:
    i = i + 1
    if i % 10 == 0:
        print(i)
    if i > fps * sec: # (대장금 영상 29 fps) 30초 처리하는데 약 40분 걸림
        break
    ret, hr_image = vid.read()
    if not(ret):
        break
    hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
    hr_image = preprocess_image(hr_image)
    
    # hr --> lr
    lr_image = downscale_image(tf.squeeze(hr_image))

    # lr --> sr1
    sr1_image = model(lr_image)
    sr1_image = tf.squeeze(sr1_image)
    sr1_image = tf.clip_by_value(sr1_image, 0, 255)
    sr1_image = tf.cast(sr1_image, tf.uint8).numpy()
    sr1_image = cv2.cvtColor(sr1_image, cv2.COLOR_BGR2RGB)

    # hr --> sr2
    sr2_image = model(hr_image)
    sr2_image = tf.squeeze(sr2_image)
    sr2_image = tf.clip_by_value(sr2_image, 0, 255)
    sr2_image = tf.cast(sr2_image, tf.uint8).numpy()
    sr2_image = cv2.cvtColor(sr2_image, cv2.COLOR_BGR2RGB)

    lr_image = tf.squeeze(lr_image)
    lr_image = tf.clip_by_value(lr_image, 0, 255)
    lr_image = tf.cast(lr_image, tf.uint8).numpy()
    lr_image = cv2.cvtColor(lr_image, cv2.COLOR_BGR2RGB)

    lr_out.write(lr_image)
    sr1_out.write(sr1_image)
    sr2_out.write(sr2_image)


vid.release()
lr_out.release()
sr1_out.release()
sr2_out.release()
cv2.destroyAllWindows()